In [8]:
import tensorflow as tf

tf.config.run_functions_eagerly(True)  # Enable eager execution mode
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler


In [9]:
# load the dataset
df = pd.read_csv('synthetic_cucumber_data.csv', delimiter=',')
df.head()

,hour,par,weight
0,2,559.885421,287.265076
1,10,784.653456,96.684831
2,10,177.388494,470.426324
3,7,1580.565045,83.389955
4,9,1831.202411,482.348470


In [10]:
# split into input (X) and output (y) variables
# Preprocess input features (time and par) and target (weight)
X = df[['hour', 'par']].values  # Using time and light as input features
y = df['weight'].values  # Target is the weight

# Scale input features for better neural network performance
scaler_X = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)

# Scale the target variable (weight) to reduce large value impact on training
scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1)).flatten()  # Reshape and flatten for compatibility

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)


In [11]:
# Build a more complex neural network model to predict weight from time and light
model = Sequential()
model.add(Dense(64, input_shape=(2,), activation='relu'))  # More neurons in first layer
model.add(Dropout(0.2))  # Add dropout to prevent overfitting
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='linear'))  # Linear activation for regression output

# Compile the model with a suitable loss function for regression
model.compile(loss='mse', optimizer='adam', metrics=['mae'])  # Using mean squared error for regression

/Users/ondrahruby/mambaforge/envs/tf_m1/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
# Add early stopping to avoid overfitting and unnecessary training
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Fit the model on the dataset
history = model.fit(
    X_train, y_train,
    epochs=100,  # Set higher epochs; early stopping will handle stopping early
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping]
)


Epoch 1/100
  1/200 ━━━━━━━━━━━━━━━━━━━━ 27s 139ms/step - loss: 1.1337 - mae: 0.9339

/Users/ondrahruby/mambaforge/envs/tf_m1/lib/python3.9/site-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


200/200 ━━━━━━━━━━━━━━━━━━━━ 11s 55ms/step - loss: 1.0064 - mae: 0.8748 - val_loss: 1.0319 - val_mae: 0.8830
Epoch 2/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 11s 53ms/step - loss: 1.0018 - mae: 0.8671 - val_loss: 1.0366 - val_mae: 0.8840
Epoch 3/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 11s 55ms/step - loss: 1.0124 - mae: 0.8750 - val_loss: 1.0303 - val_mae: 0.8830
Epoch 4/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 14s 70ms/step - loss: 0.9892 - mae: 0.8659 - val_loss: 1.0319 - val_mae: 0.8832
Epoch 5/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 11s 57ms/step - loss: 0.9937 - mae: 0.8680 - val_loss: 1.0323 - val_mae: 0.8831
Epoch 6/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 15s 73ms/step - loss: 0.9904 - mae: 0.8628 - val_loss: 1.0320 - val_mae: 0.8831
Epoch 7/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 11s 53ms/step - loss: 0.9923 - mae: 0.8657 - val_loss: 1.0321 - val_mae: 0.8830
Epoch 8/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 11s 54ms/step - loss: 0.9891 - mae: 0.8624 - val_loss: 1.0324 - val_mae: 0.8831
Epoch 9/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 10s

In [13]:
# Evaluate the model
loss, mae = model.evaluate(X_test, y_test)
print(f"Test MAE: {mae:.3f}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.9939 - mae: 0.8682
Test MAE: 0.862


In [14]:
# Predicting the weight for new data (e.g., when deciding optimal harvesting conditions)
example_time = 14  # e.g., 2 PM
example_par = 1500  # Example light value
example_input = scaler_X.transform([[example_time, example_par]])
predicted_weight_scaled = model.predict(example_input)
predicted_weight = scaler_y.inverse_transform(predicted_weight_scaled.reshape(-1, 1)).flatten()[0]
print(f"Predicted weight: {predicted_weight:.2f} grams")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted weight: 275.26 grams


/Users/ondrahruby/mambaforge/envs/tf_m1/lib/python3.9/site-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
